In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

In [2]:
pd.set_option('display.max_columns', None)
data = pd.read_csv('../data/final_data_corrected.csv')
data.head(1)

,Date,Time,TeamH,TeamA,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,HStr,HSOH,HSOA,HSAH,HSAA,HSDH,HSDA,HO,HA,HM,HD,HDP,HIP,HSAAg,HATAAg,HGls,HAssts,HGlsPK,HxG,HNpxG,HxAG,HPoss,HPrg,HPrgP,HGKS%,HGKCS%,HSoT,Hg-xG,HCmp,HCmp%,HPrgDist,HXa,HKP,HPPA,HCrsPA,HTB,HCrs,HCK,HSCA,HGCA,HTkIW,HLost,HBlocks,HPass,HInt,HClr,HAttPen,HSucc%,HCPA,HDis,HPrgR,HOnG,HOnGA,HOnxG,HOnxGA,AStr,ASOH,ASOA,ASAH,ASAA,ASDH,ASDA,AO,AA,AM,AD,ADP,AIP,ASAAg,ATAAg,AGls,AAssts,AGlsPK,AxG,ANpxG,AxAG,APoss,APrg,APrgP,AGKS%,AGKCS%,ASoT,Ag-xG,ACmp,ACmp%,APrgDist,AXa,AKP,APPA,ACrsPA,ATB,ACrs,ACK,ASCA,AGCA,ATkIW,ALost,ABlocks,APass,AInt,AClr,AAttPen,ASucc%,ACPA,ADis,APrgR,AOnG,AOnGA,AOnxG,AOnxGA
0,5/8/2022,20:00,Crystal Palace,Arsenal,0,2,A,0,1,A,A Taylor,4.2,3.6,1.85,4.33,3.5,1.87,4.3,3.55,1.85,4.5,3.65,1.89,4.4,3.5,1.83,4.6,3.5,1.87,4.6,3.78,1.95,4.39,3.59,1.88,2.1,1.72,2.14,1.78,2.19,1.91,2.09,1.76,0.5,2.04,1.89,2.03,1.89,2.06,1.91,2.01,1.87,4.5,3.6,1.8,4.5,3.5,1.83,4.4,3.55,1.85,4.58,3.63,1.88,4.8,3.4,1.78,4.75,3.5,1.85,5.01,3.7,1.91,4.56,3.57,1.85,2.1,1.72,2.14,1.78,2.19,1.91,2.08,1.76,0.5,2.09,1.84,2.04,1.88,2.09,1.88,2.03,1.85,3,1085,1100,1110,1110,1060,1090,76,77,75,76,4,2,26.6,25.2,1.0,0.76,0.97,1.02,0.96,0.75,45.8,14.2,32.2,71.8,24.3,3.51,-0.02,348.8,77.2,2310.4,0.61,8.19,6.14,1.27,0.95,15.4,4.89,19.4,1.76,11.2,6.73,12.3,9.08,9.65,21.5,19.9,46.9,5.41,12.1,32.1,1.05,1.3,1.02,1.29,4,1245,1285,1250,1250,1240,1320,80,83,81,79,7,7,24.1,23.0,2.14,1.62,2.05,1.87,1.79,1.39,59.5,21.7,54.2,70.6,35.1,5.03,0.27,482.8,83.1,2658.9,1.21,11.7,12.01,1.65,2.24,17.8,5.81,27.6,3.81,8.89,6.51,9.49,7.22,6.35,15.8,34.0,46.3,7.38,10.1,53.5,2.24,1.16,1.87,1.12


In [3]:
X = data[['Date', 'Time', 'TeamH', 'TeamA', 'FTR', 'HTR', 'Referee', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA',
          'PSH', 'PSD', 'PSA', 'WHH','WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'MaxH', 'MaxD', 'MaxA','AvgH', 'AvgD', 'AvgA', 'B365>2.5',
          'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5', 'Avg<2.5', 'AHh', 'B365AHH','B365AHA', 'PAHH', 'PAHA', 
          'MaxAHH', 'MaxAHA', "AvgAHH", 'AvgAHA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD', 'BWCA', 'IWCH', 'IWCD', 'IWCA',
          'PSCH', 'PSCD', 'PSCA', 'WHCH', 'WHCD', 'WHCA', 'VCCH', 'VCCD', 'VCCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD',
          'AvgCA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5', 'PC<2.5', 'MaxC>2.5', 'MaxC<2.5', 'AvgC>2.5', 'AvgC<2.5', 'AHCh', 'B365CAHH',
          'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA', 'AvgCAHH', 'AvgCAHA', 'HStr', 'HSOH', 'HSOA', 'HSAH', 'HSAA', 'HSDH',
          'HSDA', 'HO', 'HA', 'HM', 'HD', 'HDP', 'HIP', 'HSAAg', 'HATAAg', 'HGls', 'HAssts', 'HGlsPK', 'HxG', 'HNpxG', 'HxAG',
          'HPoss', 'HPrg', 'HPrgP', 'HGKS%', 'HGKCS%', 'HSoT', 'Hg-xG', 'HCmp', 'HCmp%', 'HPrgDist', 'HXa', 'HKP', 'HPPA', 'HCrsPA',
          'HTB', 'HCrs', 'HCK', 'HSCA', 'HGCA', 'HTkIW', 'HLost', 'HBlocks', 'HPass', 'HInt', 'HClr', 'HAttPen', 'HSucc%', 'HCPA',
          'HDis', 'HPrgR', 'HOnG', 'HOnGA', 'HOnxG', 'HOnxGA', 'AStr', 'ASOH', 'ASOA', 'ASAH', 'ASAA', 'ASDH', 'ASDA', 'AO', 'AA',
          'AM', 'AD', 'ADP', 'AIP', 'ASAAg', 'ATAAg', 'AGls', 'AAssts', 'AGlsPK', 'AxG', 'ANpxG', 'AxAG', 'APoss', 'APrg', 'APrgP',
          'AGKS%', 'AGKCS%', 'ASoT', 'Ag-xG', 'ACmp', 'ACmp%', 'APrgDist', 'AXa', 'AKP', 'APPA', 'ACrsPA', 'ATB', 'ACrs', 'ACK',
          'ASCA', 'AGCA', 'ATkIW', 'ALost', 'ABlocks', 'APass', 'AInt', 'AClr','AAttPen', 'ASucc%', 'ACPA', 'ADis', 'APrgR', 'AOnG',
          'AOnGA', 'AOnxG', 'AOnxGA']].copy()
y = data['FTHG']

In [4]:
X = X.drop(columns = ['Date', 'Time', 'FTR', 'HTR'])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321)

In [6]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse_output = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True))
    ]
)

pipe.fit(X_train, y_train)

Iteration 1, loss = 1.55501860
Iteration 2, loss = 1.10991950
Iteration 3, loss = 1.08703787
Iteration 4, loss = 0.97386319
Iteration 5, loss = 0.85842134
Iteration 6, loss = 0.83735577
Iteration 7, loss = 0.84074040
Iteration 8, loss = 0.82304228
Iteration 9, loss = 0.79163291
Iteration 10, loss = 0.77737367
Iteration 11, loss = 0.77266966
Iteration 12, loss = 0.75678876
Iteration 13, loss = 0.73127536
Iteration 14, loss = 0.71825727
Iteration 15, loss = 0.71259149
Iteration 16, loss = 0.70335758
Iteration 17, loss = 0.69024372
Iteration 18, loss = 0.68081203
Iteration 19, loss = 0.67412651
Iteration 20, loss = 0.66652008
Iteration 21, loss = 0.65842355
Iteration 22, loss = 0.65151867
Iteration 23, loss = 0.64446249
Iteration 24, loss = 0.63672880
Iteration 25, loss = 0.63131505
Iteration 26, loss = 0.62607228
Iteration 27, loss = 0.62183740
Iteration 28, loss = 0.61513694
Iteration 29, loss = 0.61030370
Iteration 30, loss = 0.60417365
Iteration 31, loss = 0.59752075
Iteration 32, los

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr', MLPRegressor(verbose=True))])

In [7]:
print(f'R2 Score: {r2_score(y_test, pipe.predict(X_test))}')
print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2 Score: -0.1856871976136767
MAE: 1.2273385466351703


In [8]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (100, 100, 100),
                            max_iter = 1000))
    ]
)

pipe.fit(X_train, y_train)

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 1, loss = 1.92026696
Iteration 2, loss = 1.19306063
Iteration 3, loss = 0.97890825
Iteration 4, loss = 1.05468223
Iteration 5, loss = 0.95843178
Iteration 6, loss = 0.86257679
Iteration 7, loss = 0.86435572
Iteration 8, loss = 0.86119256
Iteration 9, loss = 0.82317402
Iteration 10, loss = 0.77655330
Iteration 11, loss = 0.75471162
Iteration 12, loss = 0.76524652
Iteration 13, loss = 0.74874709
Iteration 14, loss = 0.72067883
Iteration 15, loss = 0.72169393
Iteration 16, loss = 0.71338234
Iteration 17, loss = 0.69555628
Iteration 18, loss = 0.68800359
Iteration 19, loss = 0.68119524
Iteration 20, loss = 0.67046083
Iteration 21, loss = 0.65592256
Iteration 22, loss = 0.65029180
Iteration 23, loss = 0.64309096
Iteration 24, loss = 0.63050888
Iteration 25, loss = 0.62075955
Iteration 26, loss = 0.60639186
Iteration 27, loss = 0.59379699
Iteration 28, loss = 0.58774981
Iteration 29, loss = 0.57714948
Iteration 30, loss = 0.55755457
Iteration 31, loss = 0.54651378
Iteration 32, los

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(hidden_layer_sizes=(100, 100, 100), max_iter=1000,
                              verbose=True))])

In [9]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: -0.2540046764413413
MAE: 1.322588249198596


In [10]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (60, 60, 60),
                            alpha = 24))
    ]
)

pipe.fit(X_train, y_train)

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 1, loss = 21.11634593
Iteration 2, loss = 20.03869475
Iteration 3, loss = 19.18168187
Iteration 4, loss = 18.43343220
Iteration 5, loss = 17.74633536
Iteration 6, loss = 17.17991024
Iteration 7, loss = 16.69533982
Iteration 8, loss = 16.25761252
Iteration 9, loss = 15.82434840
Iteration 10, loss = 15.37713199
Iteration 11, loss = 14.92504600
Iteration 12, loss = 14.48449500
Iteration 13, loss = 14.06008861
Iteration 14, loss = 13.66075908
Iteration 15, loss = 13.28043565
Iteration 16, loss = 12.90944246
Iteration 17, loss = 12.54408695
Iteration 18, loss = 12.18400787
Iteration 19, loss = 11.82822623
Iteration 20, loss = 11.48822061
Iteration 21, loss = 11.16090508
Iteration 22, loss = 10.84200872
Iteration 23, loss = 10.53586007
Iteration 24, loss = 10.23649064
Iteration 25, loss = 9.94514598
Iteration 26, loss = 9.66380740
Iteration 27, loss = 9.39138072
Iteration 28, loss = 9.12947755
Iteration 29, loss = 8.87398791
Iteration 30, loss = 8.62533337
Iteration 31, loss = 8.38

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(alpha=24, hidden_layer_sizes=(60, 60, 60),
                              verbose=True))])

In [11]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: 0.12234909263092575
MAE: 1.0702081310719975


In [12]:
X = data[['Date', 'Time', 'TeamH', 'TeamA', 'FTR', 'HTR', 'Referee', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA',
          'PSH', 'PSD', 'PSA', 'WHH','WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'MaxH', 'MaxD', 'MaxA','AvgH', 'AvgD', 'AvgA', 'B365>2.5',
          'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5', 'Avg<2.5', 'AHh', 'B365AHH','B365AHA', 'PAHH', 'PAHA', 
          'MaxAHH', 'MaxAHA', "AvgAHH", 'AvgAHA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD', 'BWCA', 'IWCH', 'IWCD', 'IWCA',
          'PSCH', 'PSCD', 'PSCA', 'WHCH', 'WHCD', 'WHCA', 'VCCH', 'VCCD', 'VCCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD',
          'AvgCA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5', 'PC<2.5', 'MaxC>2.5', 'MaxC<2.5', 'AvgC>2.5', 'AvgC<2.5', 'AHCh', 'B365CAHH',
          'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA', 'AvgCAHH', 'AvgCAHA', 'HStr', 'HSOH', 'HSOA', 'HSAH', 'HSAA', 'HSDH',
          'HSDA', 'HO', 'HA', 'HM', 'HD', 'HDP', 'HIP', 'HSAAg', 'HATAAg', 'HGls', 'HAssts', 'HGlsPK', 'HxG', 'HNpxG', 'HxAG',
          'HPoss', 'HPrg', 'HPrgP', 'HGKS%', 'HGKCS%', 'HSoT', 'Hg-xG', 'HCmp', 'HCmp%', 'HPrgDist', 'HXa', 'HKP', 'HPPA', 'HCrsPA',
          'HTB', 'HCrs', 'HCK', 'HSCA', 'HGCA', 'HTkIW', 'HLost', 'HBlocks', 'HPass', 'HInt', 'HClr', 'HAttPen', 'HSucc%', 'HCPA',
          'HDis', 'HPrgR', 'HOnG', 'HOnGA', 'HOnxG', 'HOnxGA', 'AStr', 'ASOH', 'ASOA', 'ASAH', 'ASAA', 'ASDH', 'ASDA', 'AO', 'AA',
          'AM', 'AD', 'ADP', 'AIP', 'ASAAg', 'ATAAg', 'AGls', 'AAssts', 'AGlsPK', 'AxG', 'ANpxG', 'AxAG', 'APoss', 'APrg', 'APrgP',
          'AGKS%', 'AGKCS%', 'ASoT', 'Ag-xG', 'ACmp', 'ACmp%', 'APrgDist', 'AXa', 'AKP', 'APPA', 'ACrsPA', 'ATB', 'ACrs', 'ACK',
          'ASCA', 'AGCA', 'ATkIW', 'ALost', 'ABlocks', 'APass', 'AInt', 'AClr','AAttPen', 'ASucc%', 'ACPA', 'ADis', 'APrgR', 'AOnG',
          'AOnGA', 'AOnxG', 'AOnxGA']].copy()
y = data['FTAG']

In [13]:
X = X.drop(columns = ['Date', 'Time', 'FTR', 'HTR'])

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321)

In [15]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse_output = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True))
    ]
)

pipe.fit(X_train, y_train)

Iteration 1, loss = 0.79532087
Iteration 2, loss = 0.67694661
Iteration 3, loss = 0.68276273
Iteration 4, loss = 0.62063257
Iteration 5, loss = 0.59702395
Iteration 6, loss = 0.60791907
Iteration 7, loss = 0.58978668
Iteration 8, loss = 0.56453466
Iteration 9, loss = 0.56241486
Iteration 10, loss = 0.56118206
Iteration 11, loss = 0.54378928
Iteration 12, loss = 0.53009301
Iteration 13, loss = 0.52902315
Iteration 14, loss = 0.51753340
Iteration 15, loss = 0.50774802
Iteration 16, loss = 0.50231157
Iteration 17, loss = 0.49475243
Iteration 18, loss = 0.48370358
Iteration 19, loss = 0.48100836
Iteration 20, loss = 0.47410775
Iteration 21, loss = 0.46269614
Iteration 22, loss = 0.45931835
Iteration 23, loss = 0.45842426
Iteration 24, loss = 0.45035888
Iteration 25, loss = 0.44586915
Iteration 26, loss = 0.43821003
Iteration 27, loss = 0.43489455
Iteration 28, loss = 0.42909796
Iteration 29, loss = 0.42448444
Iteration 30, loss = 0.41825841
Iteration 31, loss = 0.41397857
Iteration 32, los

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr', MLPRegressor(verbose=True))])

In [16]:
print(f'R2 Score: {r2_score(y_test, pipe.predict(X_test))}')
print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2 Score: -0.41438990427303524
MAE: 1.063311861160067


In [17]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (100, 100, 100),
                            max_iter = 1000))
    ]
)

pipe.fit(X_train, y_train)

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 1, loss = 1.50251752
Iteration 2, loss = 0.87320438
Iteration 3, loss = 0.67480837
Iteration 4, loss = 0.77371345
Iteration 5, loss = 0.73480775
Iteration 6, loss = 0.63384601
Iteration 7, loss = 0.61353001
Iteration 8, loss = 0.64285819
Iteration 9, loss = 0.64385076
Iteration 10, loss = 0.61103829
Iteration 11, loss = 0.58026444
Iteration 12, loss = 0.58749650
Iteration 13, loss = 0.57875891
Iteration 14, loss = 0.56302188
Iteration 15, loss = 0.55237829
Iteration 16, loss = 0.54861994
Iteration 17, loss = 0.54313786
Iteration 18, loss = 0.53063929
Iteration 19, loss = 0.52666625
Iteration 20, loss = 0.52594878
Iteration 21, loss = 0.51217679
Iteration 22, loss = 0.50230111
Iteration 23, loss = 0.50132813
Iteration 24, loss = 0.49177235
Iteration 25, loss = 0.47842165
Iteration 26, loss = 0.47261221
Iteration 27, loss = 0.46453254
Iteration 28, loss = 0.45253421
Iteration 29, loss = 0.44325927
Iteration 30, loss = 0.43272415
Iteration 31, loss = 0.42115395
Iteration 32, los

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(hidden_layer_sizes=(100, 100, 100), max_iter=1000,
                              verbose=True))])

In [18]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: -0.4784722836486597
MAE: 1.1146203473240428


In [19]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (60, 60, 60),
                            alpha = 24))
    ]
)

pipe.fit(X_train, y_train)

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 1, loss = 19.31425227
Iteration 2, loss = 18.58302387
Iteration 3, loss = 17.98168921
Iteration 4, loss = 17.46184320
Iteration 5, loss = 16.95932791
Iteration 6, loss = 16.45449064
Iteration 7, loss = 15.95300323
Iteration 8, loss = 15.46115426
Iteration 9, loss = 14.98767249
Iteration 10, loss = 14.52965294
Iteration 11, loss = 14.08627177
Iteration 12, loss = 13.65367965
Iteration 13, loss = 13.23143220
Iteration 14, loss = 12.81949517
Iteration 15, loss = 12.41890764
Iteration 16, loss = 12.03128483
Iteration 17, loss = 11.65733951
Iteration 18, loss = 11.29480395
Iteration 19, loss = 10.94370207
Iteration 20, loss = 10.60299672
Iteration 21, loss = 10.27285674
Iteration 22, loss = 9.95226875
Iteration 23, loss = 9.64241116
Iteration 24, loss = 9.34333622
Iteration 25, loss = 9.05407950
Iteration 26, loss = 8.77361028
Iteration 27, loss = 8.50243842
Iteration 28, loss = 8.24025051
Iteration 29, loss = 7.98677443
Iteration 30, loss = 7.74127087
Iteration 31, loss = 7.50400

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(alpha=24, hidden_layer_sizes=(60, 60, 60),
                              verbose=True))])

In [20]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: 0.01749437903072537
MAE: 0.9318830342065986


In [21]:
pd.set_option('display.max_columns', None)
data1 = pd.read_csv('../data/data_betting.csv')
data1.head(1)

,Unnamed: 0,Date,Time,TeamH,TeamA,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,0,5/8/2022,20:00,Crystal Palace,Arsenal,0,2,A,0,1,A,A Taylor,0,0,4.2,3.6,1.85,4.33,3.5,1.87,4.3,3.55,1.85,4.5,3.65,1.89,4.4,3.5,1.83,4.6,3.5,1.87,4.6,3.78,1.95,4.39,3.59,1.88,2.1,1.72,2.14,1.78,2.19,1.91,2.09,1.76,0.5,2.04,1.89,2.03,1.89,2.06,1.91,2.01,1.87,4.5,3.6,1.8,4.5,3.5,1.83,4.4,3.55,1.85,4.58,3.63,1.88,4.8,3.4,1.78,4.75,3.5,1.85,5.01,3.7,1.91,4.56,3.57,1.85,2.1,1.72,2.14,1.78,2.19,1.91,2.08,1.76,0.5,2.09,1.84,2.04,1.88,2.09,1.88,2.03,1.85


In [22]:
X = data1[['Date', 'Time', 'TeamH', 'TeamA', 'FTR', 'HTR', 'Referee', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA',
          'PSH', 'PSD', 'PSA', 'WHH','WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'MaxH', 'MaxD', 'MaxA','AvgH', 'AvgD', 'AvgA', 'B365>2.5',
          'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5', 'Avg<2.5', 'AHh', 'B365AHH','B365AHA', 'PAHH', 'PAHA', 
          'MaxAHH', 'MaxAHA', "AvgAHH", 'AvgAHA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD', 'BWCA', 'IWCH', 'IWCD', 'IWCA',
          'PSCH', 'PSCD', 'PSCA', 'WHCH', 'WHCD', 'WHCA', 'VCCH', 'VCCD', 'VCCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD',
          'AvgCA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5', 'PC<2.5', 'MaxC>2.5', 'MaxC<2.5', 'AvgC>2.5', 'AvgC<2.5', 'AHCh', 'B365CAHH',
          'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA', 'AvgCAHH', 'AvgCAHA']].copy()
y = data1['FTHG'] 

In [23]:
X = X.drop(columns = ['Date', 'Time', 'FTR', 'HTR'])

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321)

In [25]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse_output = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True))
    ]
)

pipe.fit(X_train, y_train)

Iteration 1, loss = 2.63774676
Iteration 2, loss = 2.02666592
Iteration 3, loss = 1.57818322
Iteration 4, loss = 1.25187519
Iteration 5, loss = 1.03485893
Iteration 6, loss = 0.93122724
Iteration 7, loss = 0.90677772
Iteration 8, loss = 0.90720594
Iteration 9, loss = 0.91288749
Iteration 10, loss = 0.91445973
Iteration 11, loss = 0.89584128
Iteration 12, loss = 0.86206418
Iteration 13, loss = 0.82214814
Iteration 14, loss = 0.78765142
Iteration 15, loss = 0.76192896
Iteration 16, loss = 0.74283057
Iteration 17, loss = 0.73318938
Iteration 18, loss = 0.72747489
Iteration 19, loss = 0.72237597
Iteration 20, loss = 0.71647864
Iteration 21, loss = 0.70954070
Iteration 22, loss = 0.70155300
Iteration 23, loss = 0.69278563
Iteration 24, loss = 0.68734971
Iteration 25, loss = 0.67936098
Iteration 26, loss = 0.67555211
Iteration 27, loss = 0.67049945
Iteration 28, loss = 0.66621735
Iteration 29, loss = 0.66194985
Iteration 30, loss = 0.65749930
Iteration 31, loss = 0.65325544
Iteration 32, los

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr', MLPRegressor(verbose=True))])

In [26]:
print(f'R2 Score: {r2_score(y_test, pipe.predict(X_test))}')
print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2 Score: -0.2146162970043708
MAE: 1.2188569471177622


In [27]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (100, 100, 100),
                            max_iter = 1000))
    ]
)

pipe.fit(X_train, y_train)

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 1, loss = 1.82666208
Iteration 2, loss = 1.36756118
Iteration 3, loss = 1.07735303
Iteration 4, loss = 0.93876164
Iteration 5, loss = 0.93490384
Iteration 6, loss = 0.94247276
Iteration 7, loss = 0.91283649
Iteration 8, loss = 0.85059866
Iteration 9, loss = 0.80238055
Iteration 10, loss = 0.77804352
Iteration 11, loss = 0.76722171
Iteration 12, loss = 0.75633999
Iteration 13, loss = 0.73841150
Iteration 14, loss = 0.72310298
Iteration 15, loss = 0.71021713
Iteration 16, loss = 0.71264659
Iteration 17, loss = 0.70552359
Iteration 18, loss = 0.69245686
Iteration 19, loss = 0.68081027
Iteration 20, loss = 0.67018947
Iteration 21, loss = 0.66044341
Iteration 22, loss = 0.65160388
Iteration 23, loss = 0.63980495
Iteration 24, loss = 0.62804648
Iteration 25, loss = 0.62469891
Iteration 26, loss = 0.61492151
Iteration 27, loss = 0.59910482
Iteration 28, loss = 0.58515943
Iteration 29, loss = 0.57611016
Iteration 30, loss = 0.56181507
Iteration 31, loss = 0.54825608
Iteration 32, los

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(hidden_layer_sizes=(100, 100, 100), max_iter=1000,
                              verbose=True))])

In [28]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: -0.31617201686345764
MAE: 1.2953839280219352


In [29]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (60, 60, 60),
                            alpha = 24))
    ]
)

pipe.fit(X_train, y_train)

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 1, loss = 20.10761081
Iteration 2, loss = 19.31178285
Iteration 3, loss = 18.59745779
Iteration 4, loss = 17.94584993
Iteration 5, loss = 17.34101469
Iteration 6, loss = 16.77592411
Iteration 7, loss = 16.25607983
Iteration 8, loss = 15.77983253
Iteration 9, loss = 15.34912500
Iteration 10, loss = 14.96068138
Iteration 11, loss = 14.58926279
Iteration 12, loss = 14.22749342
Iteration 13, loss = 13.86179576
Iteration 14, loss = 13.49236671
Iteration 15, loss = 13.12938647
Iteration 16, loss = 12.77628235
Iteration 17, loss = 12.43569118
Iteration 18, loss = 12.10977038
Iteration 19, loss = 11.79581587
Iteration 20, loss = 11.49062344
Iteration 21, loss = 11.19301909
Iteration 22, loss = 10.90163632
Iteration 23, loss = 10.61406850
Iteration 24, loss = 10.33479295
Iteration 25, loss = 10.06201210
Iteration 26, loss = 9.79789779
Iteration 27, loss = 9.54179562
Iteration 28, loss = 9.29206119
Iteration 29, loss = 9.04987314
Iteration 30, loss = 8.81458218
Iteration 31, loss = 8.5

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(alpha=24, hidden_layer_sizes=(60, 60, 60),
                              verbose=True))])

In [30]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: 0.05397520230519326
MAE: 1.1218069927962886


In [31]:
X = data1[['Date', 'Time', 'TeamH', 'TeamA', 'FTR', 'HTR', 'Referee', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA',
          'PSH', 'PSD', 'PSA', 'WHH','WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'MaxH', 'MaxD', 'MaxA','AvgH', 'AvgD', 'AvgA', 'B365>2.5',
          'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5', 'Avg<2.5', 'AHh', 'B365AHH','B365AHA', 'PAHH', 'PAHA', 
          'MaxAHH', 'MaxAHA', "AvgAHH", 'AvgAHA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD', 'BWCA', 'IWCH', 'IWCD', 'IWCA',
          'PSCH', 'PSCD', 'PSCA', 'WHCH', 'WHCD', 'WHCA', 'VCCH', 'VCCD', 'VCCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD',
          'AvgCA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5', 'PC<2.5', 'MaxC>2.5', 'MaxC<2.5', 'AvgC>2.5', 'AvgC<2.5', 'AHCh', 'B365CAHH',
          'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA', 'AvgCAHH', 'AvgCAHA']].copy()
y = data1['FTAG'] 

In [32]:
X = X.drop(columns = ['Date', 'Time', 'FTR', 'HTR'])

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321)

In [34]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse_output = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True))
    ]
)

pipe.fit(X_train, y_train)

Iteration 1, loss = 1.20849784
Iteration 2, loss = 0.92910997
Iteration 3, loss = 0.78157004
Iteration 4, loss = 0.71406105
Iteration 5, loss = 0.70927792
Iteration 6, loss = 0.71069384
Iteration 7, loss = 0.70283126
Iteration 8, loss = 0.67643946
Iteration 9, loss = 0.64303845
Iteration 10, loss = 0.61313471
Iteration 11, loss = 0.59860403
Iteration 12, loss = 0.59207539
Iteration 13, loss = 0.58822858
Iteration 14, loss = 0.58373068
Iteration 15, loss = 0.57485598
Iteration 16, loss = 0.56518032
Iteration 17, loss = 0.55465598
Iteration 18, loss = 0.54557981
Iteration 19, loss = 0.53896014
Iteration 20, loss = 0.53267638
Iteration 21, loss = 0.52652250
Iteration 22, loss = 0.51983934
Iteration 23, loss = 0.51334352
Iteration 24, loss = 0.50737451
Iteration 25, loss = 0.50164680
Iteration 26, loss = 0.49593829
Iteration 27, loss = 0.49039835
Iteration 28, loss = 0.48426019
Iteration 29, loss = 0.47828949
Iteration 30, loss = 0.47355776
Iteration 31, loss = 0.46839158
Iteration 32, los

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr', MLPRegressor(verbose=True))])

In [35]:
print(f'R2 Score: {r2_score(y_test, pipe.predict(X_test))}')
print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2 Score: -0.39878364132256405
MAE: 1.0969206012530108


In [36]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (100, 100, 100),
                            max_iter = 1000))
    ]
)

pipe.fit(X_train, y_train)

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 1, loss = 1.40776398
Iteration 2, loss = 1.13901524
Iteration 3, loss = 0.92975420
Iteration 4, loss = 0.77394160
Iteration 5, loss = 0.71698750
Iteration 6, loss = 0.72701724
Iteration 7, loss = 0.73461893
Iteration 8, loss = 0.69845081
Iteration 9, loss = 0.65349156
Iteration 10, loss = 0.63079464
Iteration 11, loss = 0.62146109
Iteration 12, loss = 0.62001377
Iteration 13, loss = 0.61319835
Iteration 14, loss = 0.60099570
Iteration 15, loss = 0.58278387
Iteration 16, loss = 0.57049038
Iteration 17, loss = 0.56496424
Iteration 18, loss = 0.56039135
Iteration 19, loss = 0.55264269
Iteration 20, loss = 0.53846767
Iteration 21, loss = 0.52653966
Iteration 22, loss = 0.51616357
Iteration 23, loss = 0.50636012
Iteration 24, loss = 0.49268033
Iteration 25, loss = 0.48054685
Iteration 26, loss = 0.46842131
Iteration 27, loss = 0.45543365
Iteration 28, loss = 0.44339995
Iteration 29, loss = 0.42813456
Iteration 30, loss = 0.41338329
Iteration 31, loss = 0.40026886
Iteration 32, los

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(hidden_layer_sizes=(100, 100, 100), max_iter=1000,
                              verbose=True))])

In [37]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')
print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: -0.5638861658433441
MAE: 1.1447545342414556


In [40]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (60, 60, 60),
                            alpha = 24))
    ]
)

pipe.fit(X_train, y_train)

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 1, loss = 18.86246024
Iteration 2, loss = 18.27833928
Iteration 3, loss = 17.73724909
Iteration 4, loss = 17.21690754
Iteration 5, loss = 16.71368655
Iteration 6, loss = 16.23033984
Iteration 7, loss = 15.75774119
Iteration 8, loss = 15.30176273
Iteration 9, loss = 14.85859561
Iteration 10, loss = 14.42986647
Iteration 11, loss = 14.01543688
Iteration 12, loss = 13.61212716
Iteration 13, loss = 13.22933979
Iteration 14, loss = 12.85632735
Iteration 15, loss = 12.49621376
Iteration 16, loss = 12.14849484
Iteration 17, loss = 11.80923079
Iteration 18, loss = 11.48071501
Iteration 19, loss = 11.16016283
Iteration 20, loss = 10.84784084
Iteration 21, loss = 10.54357301
Iteration 22, loss = 10.24827428
Iteration 23, loss = 9.96136257
Iteration 24, loss = 9.68282130
Iteration 25, loss = 9.41219461
Iteration 26, loss = 9.14960030
Iteration 27, loss = 8.89467911
Iteration 28, loss = 8.64673716
Iteration 29, loss = 8.40589909
Iteration 30, loss = 8.17208374
Iteration 31, loss = 7.9448

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(alpha=24, hidden_layer_sizes=(60, 60, 60),
                              verbose=True))])

In [41]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')
print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: -0.02611626564280023
MAE: 0.9400140836650767


In [42]:
pd.set_option('display.max_columns', None)
data2 = pd.read_csv('../data/data_teams.csv')
data2.head(1)

,Date,Time,TeamH,TeamA,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HStr,HSOH,HSOA,HSAH,HSAA,HSDH,HSDA,HO,HA,HM,HD,HDP,HIP,HSAAg,HATAAg,HGls,HAssts,HGlsPK,HxG,HNpxG,HxAG,HPoss,HPrg,HPrgP,HGKS%,HGKCS%,HSoT,Hg-xG,HCmp,HCmp%,HPrgDist,HXa,HKP,HPPA,HCrsPA,HTB,HCrs,HCK,HSCA,HGCA,HTkIW,HLost,HBlocks,HPass,HInt,HClr,HAttPen,HSucc%,HCPA,HDis,HPrgR,HOnG,HOnGA,HOnxG,HOnxGA,AStr,ASOH,ASOA,ASAH,ASAA,ASDH,ASDA,AO,AA,AM,AD,ADP,AIP,ASAAg,ATAAg,AGls,AAssts,AGlsPK,AxG,ANpxG,AxAG,APoss,APrg,APrgP,AGKS%,AGKCS%,ASoT,Ag-xG,ACmp,ACmp%,APrgDist,AXa,AKP,APPA,ACrsPA,ATB,ACrs,ACK,ASCA,AGCA,ATkIW,ALost,ABlocks,APass,AInt,AClr,AAttPen,ASucc%,ACPA,ADis,APrgR,AOnG,AOnGA,AOnxG,AOnxGA
0,5/8/2022,20:00,Crystal Palace,Arsenal,0,2,A,0,1,A,A Taylor,3,1085,1100,1110,1110,1060,1090,76,77,75,76,4,2,26.6,25.2,1.0,0.76,0.97,1.02,0.96,0.75,45.8,14.2,32.2,71.8,24.3,3.51,-0.02,348.8,77.2,2310.4,0.61,8.19,6.14,1.27,0.95,15.4,4.89,19.4,1.76,11.2,6.73,12.3,9.08,9.65,21.5,19.9,46.9,5.41,12.1,32.1,1.05,1.3,1.02,1.29,4,1245,1285,1250,1250,1240,1320,80,83,81,79,7,7,24.1,23.0,2.14,1.62,2.05,1.87,1.79,1.39,59.5,21.7,54.2,70.6,35.1,5.03,0.27,482.8,83.1,2658.9,1.21,11.7,12.01,1.65,2.24,17.8,5.81,27.6,3.81,8.89,6.51,9.49,7.22,6.35,15.8,34.0,46.3,7.38,10.1,53.5,2.24,1.16,1.87,1.12


In [43]:
X = data2[['Date', 'Time', 'TeamH', 'TeamA', 'FTR', 'HTR', 'Referee', 'HStr', 'HSOH', 'HSOA', 'HSAH', 'HSAA', 'HSDH',
          'HSDA', 'HO', 'HA', 'HM', 'HD', 'HDP', 'HIP', 'HSAAg', 'HATAAg', 'HGls', 'HAssts', 'HGlsPK', 'HxG', 'HNpxG', 'HxAG',
          'HPoss', 'HPrg', 'HPrgP', 'HGKS%', 'HGKCS%', 'HSoT', 'Hg-xG', 'HCmp', 'HCmp%', 'HPrgDist', 'HXa', 'HKP', 'HPPA', 'HCrsPA',
          'HTB', 'HCrs', 'HCK', 'HSCA', 'HGCA', 'HTkIW', 'HLost', 'HBlocks', 'HPass', 'HInt', 'HClr', 'HAttPen', 'HSucc%', 'HCPA',
          'HDis', 'HPrgR', 'HOnG', 'HOnGA', 'HOnxG', 'HOnxGA', 'AStr', 'ASOH', 'ASOA', 'ASAH', 'ASAA', 'ASDH', 'ASDA', 'AO', 'AA',
          'AM', 'AD', 'ADP', 'AIP', 'ASAAg', 'ATAAg', 'AGls', 'AAssts', 'AGlsPK', 'AxG', 'ANpxG', 'AxAG', 'APoss', 'APrg', 'APrgP',
          'AGKS%', 'AGKCS%', 'ASoT', 'Ag-xG', 'ACmp', 'ACmp%', 'APrgDist', 'AXa', 'AKP', 'APPA', 'ACrsPA', 'ATB', 'ACrs', 'ACK',
          'ASCA', 'AGCA', 'ATkIW', 'ALost', 'ABlocks', 'APass', 'AInt', 'AClr','AAttPen', 'ASucc%', 'ACPA', 'ADis', 'APrgR', 'AOnG',
          'AOnGA', 'AOnxG', 'AOnxGA']].copy()
y = data2['FTHG']

In [44]:
X = X.drop(columns = ['Date', 'Time', 'FTR', 'HTR'])

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321)

In [46]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse_output = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True))
    ]
)

pipe.fit(X_train, y_train)

Iteration 1, loss = 2.77896664
Iteration 2, loss = 1.75661056
Iteration 3, loss = 1.11752522
Iteration 4, loss = 0.88308195
Iteration 5, loss = 0.90365611
Iteration 6, loss = 1.00192243
Iteration 7, loss = 1.03500917
Iteration 8, loss = 0.98368074
Iteration 9, loss = 0.89574300
Iteration 10, loss = 0.83208494
Iteration 11, loss = 0.79377969
Iteration 12, loss = 0.77941299
Iteration 13, loss = 0.78385089
Iteration 14, loss = 0.78975334
Iteration 15, loss = 0.78746033
Iteration 16, loss = 0.77816688
Iteration 17, loss = 0.76428676
Iteration 18, loss = 0.75051630
Iteration 19, loss = 0.74185404
Iteration 20, loss = 0.73737522
Iteration 21, loss = 0.73221790
Iteration 22, loss = 0.73050906
Iteration 23, loss = 0.72506075
Iteration 24, loss = 0.71806445
Iteration 25, loss = 0.71036055
Iteration 26, loss = 0.70528034
Iteration 27, loss = 0.70059044
Iteration 28, loss = 0.69697276
Iteration 29, loss = 0.69219234
Iteration 30, loss = 0.68580209
Iteration 31, loss = 0.68117304
Iteration 32, los

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr', MLPRegressor(verbose=True))])

In [47]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')
print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: -0.1419476883494113
MAE: 1.1842239096062004


In [48]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (100, 100, 100),
                            max_iter = 1000))
    ]
)

pipe.fit(X_train, y_train)

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 1, loss = 2.72688532
Iteration 2, loss = 2.00251561
Iteration 3, loss = 1.46837720
Iteration 4, loss = 1.10286717
Iteration 5, loss = 1.01158726
Iteration 6, loss = 1.07827294
Iteration 7, loss = 1.05483163
Iteration 8, loss = 0.93718809
Iteration 9, loss = 0.85988385
Iteration 10, loss = 0.84954913
Iteration 11, loss = 0.85870639
Iteration 12, loss = 0.84486913
Iteration 13, loss = 0.81134932
Iteration 14, loss = 0.77827137
Iteration 15, loss = 0.77908188
Iteration 16, loss = 0.78029987
Iteration 17, loss = 0.76735178
Iteration 18, loss = 0.74794874
Iteration 19, loss = 0.73395819
Iteration 20, loss = 0.72739196
Iteration 21, loss = 0.71882966
Iteration 22, loss = 0.70795153
Iteration 23, loss = 0.69747412
Iteration 24, loss = 0.68713839
Iteration 25, loss = 0.67657608
Iteration 26, loss = 0.66593626
Iteration 27, loss = 0.65682269
Iteration 28, loss = 0.64532652
Iteration 29, loss = 0.63257804
Iteration 30, loss = 0.62285863
Iteration 31, loss = 0.61150031
Iteration 32, los

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(hidden_layer_sizes=(100, 100, 100), max_iter=1000,
                              verbose=True))])

In [49]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: -0.34340433538408877
MAE: 1.3580885512224763


In [50]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (60, 60, 60),
                            alpha = 24))
    ]
)

pipe.fit(X_train, y_train)

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 1, loss = 19.53050690
Iteration 2, loss = 18.71299301
Iteration 3, loss = 18.06645387
Iteration 4, loss = 17.49446115
Iteration 5, loss = 17.03310416
Iteration 6, loss = 16.63753067
Iteration 7, loss = 16.22425552
Iteration 8, loss = 15.78767852
Iteration 9, loss = 15.34197575
Iteration 10, loss = 14.91518239
Iteration 11, loss = 14.51665789
Iteration 12, loss = 14.13430067
Iteration 13, loss = 13.76549342
Iteration 14, loss = 13.39883172
Iteration 15, loss = 13.03545524
Iteration 16, loss = 12.67811406
Iteration 17, loss = 12.32866845
Iteration 18, loss = 11.99078585
Iteration 19, loss = 11.66630980
Iteration 20, loss = 11.35062142
Iteration 21, loss = 11.04342855
Iteration 22, loss = 10.74388788
Iteration 23, loss = 10.45073990
Iteration 24, loss = 10.16715219
Iteration 25, loss = 9.89083757
Iteration 26, loss = 9.62477207
Iteration 27, loss = 9.36491538
Iteration 28, loss = 9.11073071
Iteration 29, loss = 8.86521045
Iteration 30, loss = 8.62431535
Iteration 31, loss = 8.39

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(alpha=24, hidden_layer_sizes=(60, 60, 60),
                              verbose=True))])

In [51]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: 0.16350328616363996
MAE: 1.056411983152845


In [52]:
X = data2[['Date', 'Time', 'TeamH', 'TeamA', 'FTR', 'HTR', 'Referee', 'HStr', 'HSOH', 'HSOA', 'HSAH', 'HSAA', 'HSDH',
          'HSDA', 'HO', 'HA', 'HM', 'HD', 'HDP', 'HIP', 'HSAAg', 'HATAAg', 'HGls', 'HAssts', 'HGlsPK', 'HxG', 'HNpxG', 'HxAG',
          'HPoss', 'HPrg', 'HPrgP', 'HGKS%', 'HGKCS%', 'HSoT', 'Hg-xG', 'HCmp', 'HCmp%', 'HPrgDist', 'HXa', 'HKP', 'HPPA', 'HCrsPA',
          'HTB', 'HCrs', 'HCK', 'HSCA', 'HGCA', 'HTkIW', 'HLost', 'HBlocks', 'HPass', 'HInt', 'HClr', 'HAttPen', 'HSucc%', 'HCPA',
          'HDis', 'HPrgR', 'HOnG', 'HOnGA', 'HOnxG', 'HOnxGA', 'AStr', 'ASOH', 'ASOA', 'ASAH', 'ASAA', 'ASDH', 'ASDA', 'AO', 'AA',
          'AM', 'AD', 'ADP', 'AIP', 'ASAAg', 'ATAAg', 'AGls', 'AAssts', 'AGlsPK', 'AxG', 'ANpxG', 'AxAG', 'APoss', 'APrg', 'APrgP',
          'AGKS%', 'AGKCS%', 'ASoT', 'Ag-xG', 'ACmp', 'ACmp%', 'APrgDist', 'AXa', 'AKP', 'APPA', 'ACrsPA', 'ATB', 'ACrs', 'ACK',
          'ASCA', 'AGCA', 'ATkIW', 'ALost', 'ABlocks', 'APass', 'AInt', 'AClr','AAttPen', 'ASucc%', 'ACPA', 'ADis', 'APrgR', 'AOnG',
          'AOnGA', 'AOnxG', 'AOnxGA']].copy()
y = data2['FTAG']

In [53]:
X = X.drop(columns = ['Date', 'Time', 'FTR', 'HTR'])

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321)

In [55]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse_output = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True))
    ]
)

pipe.fit(X_train, y_train)

Iteration 1, loss = 0.86518318
Iteration 2, loss = 0.72473490
Iteration 3, loss = 0.64718646
Iteration 4, loss = 0.59825893
Iteration 5, loss = 0.57620814
Iteration 6, loss = 0.56724195
Iteration 7, loss = 0.56637888
Iteration 8, loss = 0.56064457
Iteration 9, loss = 0.54871053
Iteration 10, loss = 0.53501961
Iteration 11, loss = 0.51898657
Iteration 12, loss = 0.50811023
Iteration 13, loss = 0.50000927
Iteration 14, loss = 0.49373788
Iteration 15, loss = 0.48871598
Iteration 16, loss = 0.48282734
Iteration 17, loss = 0.47532898
Iteration 18, loss = 0.46902645
Iteration 19, loss = 0.46082872
Iteration 20, loss = 0.45687578
Iteration 21, loss = 0.45287341
Iteration 22, loss = 0.44792471
Iteration 23, loss = 0.44680737
Iteration 24, loss = 0.43957836
Iteration 25, loss = 0.43397636
Iteration 26, loss = 0.43177911
Iteration 27, loss = 0.42720089
Iteration 28, loss = 0.42363009
Iteration 29, loss = 0.42000359
Iteration 30, loss = 0.41583876
Iteration 31, loss = 0.41237216
Iteration 32, los

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr', MLPRegressor(verbose=True))])

In [56]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: -0.22465801687163056
MAE: 1.037123807635497


In [57]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (100, 100, 100),
                            max_iter = 1000))
    ]
)

pipe.fit(X_train, y_train)

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 1, loss = 1.08659745
Iteration 2, loss = 0.73585936
Iteration 3, loss = 0.70163302
Iteration 4, loss = 0.66413877
Iteration 5, loss = 0.60513741
Iteration 6, loss = 0.57763336
Iteration 7, loss = 0.57255338
Iteration 8, loss = 0.55521805
Iteration 9, loss = 0.54555559
Iteration 10, loss = 0.54917777
Iteration 11, loss = 0.53973343
Iteration 12, loss = 0.52220502
Iteration 13, loss = 0.50848324
Iteration 14, loss = 0.50140387
Iteration 15, loss = 0.49035172
Iteration 16, loss = 0.48639165
Iteration 17, loss = 0.47372906
Iteration 18, loss = 0.46362059
Iteration 19, loss = 0.45148225
Iteration 20, loss = 0.43959495
Iteration 21, loss = 0.43486451
Iteration 22, loss = 0.42133946
Iteration 23, loss = 0.41410122
Iteration 24, loss = 0.41209860
Iteration 25, loss = 0.39074960
Iteration 26, loss = 0.38332986
Iteration 27, loss = 0.36803486
Iteration 28, loss = 0.35833469
Iteration 29, loss = 0.35038672
Iteration 30, loss = 0.33708637
Iteration 31, loss = 0.32510968
Iteration 32, los

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(hidden_layer_sizes=(100, 100, 100), max_iter=1000,
                              verbose=True))])

In [58]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: -0.33187254372557806
MAE: 1.084872805633458


In [59]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (60, 60, 60),
                            alpha = 24))
    ]
)

pipe.fit(X_train, y_train)

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 1, loss = 19.14145788
Iteration 2, loss = 18.47106996
Iteration 3, loss = 17.86506526
Iteration 4, loss = 17.30720320
Iteration 5, loss = 16.78868345
Iteration 6, loss = 16.30083904
Iteration 7, loss = 15.84645179
Iteration 8, loss = 15.40923473
Iteration 9, loss = 14.98957183
Iteration 10, loss = 14.57343584
Iteration 11, loss = 14.16267425
Iteration 12, loss = 13.75759751
Iteration 13, loss = 13.36509191
Iteration 14, loss = 12.98534799
Iteration 15, loss = 12.61720085
Iteration 16, loss = 12.26019149
Iteration 17, loss = 11.91209351
Iteration 18, loss = 11.57253176
Iteration 19, loss = 11.24288092
Iteration 20, loss = 10.92121434
Iteration 21, loss = 10.60886505
Iteration 22, loss = 10.30580782
Iteration 23, loss = 10.01069544
Iteration 24, loss = 9.72445970
Iteration 25, loss = 9.44700375
Iteration 26, loss = 9.17812459
Iteration 27, loss = 8.91601454
Iteration 28, loss = 8.66202281
Iteration 29, loss = 8.41542521
Iteration 30, loss = 8.17576936
Iteration 31, loss = 7.944

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(alpha=24, hidden_layer_sizes=(60, 60, 60),
                              verbose=True))])

In [60]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: 0.029089953355725173
MAE: 0.935757448549554


In [61]:
pd.set_option('display.max_columns', None)
data3 = pd.read_csv('../data/final_data_corrected.csv')
data3.head(1)

,Date,Time,TeamH,TeamA,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,HStr,HSOH,HSOA,HSAH,HSAA,HSDH,HSDA,HO,HA,HM,HD,HDP,HIP,HSAAg,HATAAg,HGls,HAssts,HGlsPK,HxG,HNpxG,HxAG,HPoss,HPrg,HPrgP,HGKS%,HGKCS%,HSoT,Hg-xG,HCmp,HCmp%,HPrgDist,HXa,HKP,HPPA,HCrsPA,HTB,HCrs,HCK,HSCA,HGCA,HTkIW,HLost,HBlocks,HPass,HInt,HClr,HAttPen,HSucc%,HCPA,HDis,HPrgR,HOnG,HOnGA,HOnxG,HOnxGA,AStr,ASOH,ASOA,ASAH,ASAA,ASDH,ASDA,AO,AA,AM,AD,ADP,AIP,ASAAg,ATAAg,AGls,AAssts,AGlsPK,AxG,ANpxG,AxAG,APoss,APrg,APrgP,AGKS%,AGKCS%,ASoT,Ag-xG,ACmp,ACmp%,APrgDist,AXa,AKP,APPA,ACrsPA,ATB,ACrs,ACK,ASCA,AGCA,ATkIW,ALost,ABlocks,APass,AInt,AClr,AAttPen,ASucc%,ACPA,ADis,APrgR,AOnG,AOnGA,AOnxG,AOnxGA
0,5/8/2022,20:00,Crystal Palace,Arsenal,0,2,A,0,1,A,A Taylor,4.2,3.6,1.85,4.33,3.5,1.87,4.3,3.55,1.85,4.5,3.65,1.89,4.4,3.5,1.83,4.6,3.5,1.87,4.6,3.78,1.95,4.39,3.59,1.88,2.1,1.72,2.14,1.78,2.19,1.91,2.09,1.76,0.5,2.04,1.89,2.03,1.89,2.06,1.91,2.01,1.87,4.5,3.6,1.8,4.5,3.5,1.83,4.4,3.55,1.85,4.58,3.63,1.88,4.8,3.4,1.78,4.75,3.5,1.85,5.01,3.7,1.91,4.56,3.57,1.85,2.1,1.72,2.14,1.78,2.19,1.91,2.08,1.76,0.5,2.09,1.84,2.04,1.88,2.09,1.88,2.03,1.85,3,1085,1100,1110,1110,1060,1090,76,77,75,76,4,2,26.6,25.2,1.0,0.76,0.97,1.02,0.96,0.75,45.8,14.2,32.2,71.8,24.3,3.51,-0.02,348.8,77.2,2310.4,0.61,8.19,6.14,1.27,0.95,15.4,4.89,19.4,1.76,11.2,6.73,12.3,9.08,9.65,21.5,19.9,46.9,5.41,12.1,32.1,1.05,1.3,1.02,1.29,4,1245,1285,1250,1250,1240,1320,80,83,81,79,7,7,24.1,23.0,2.14,1.62,2.05,1.87,1.79,1.39,59.5,21.7,54.2,70.6,35.1,5.03,0.27,482.8,83.1,2658.9,1.21,11.7,12.01,1.65,2.24,17.8,5.81,27.6,3.81,8.89,6.51,9.49,7.22,6.35,15.8,34.0,46.3,7.38,10.1,53.5,2.24,1.16,1.87,1.12


In [ ]:
data_basic = data3.drop(['B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA',
          'PSH', 'PSD', 'PSA', 'WHH','WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'MaxH', 'MaxD', 'MaxA','AvgH', 'AvgD', 'AvgA', 'B365>2.5',
          'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5', 'Avg<2.5', 'AHh', 'B365AHH','B365AHA', 'PAHH', 'PAHA', 
          'MaxAHH', 'MaxAHA', "AvgAHH", 'AvgAHA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD', 'BWCA', 'IWCH', 'IWCD', 'IWCA',
          'PSCH', 'PSCD', 'PSCA', 'WHCH', 'WHCD', 'WHCA', 'VCCH', 'VCCD', 'VCCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD',
          'AvgCA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5', 'PC<2.5', 'MaxC>2.5', 'MaxC<2.5', 'AvgC>2.5', 'AvgC<2.5', 'AHCh', 'B365CAHH',
          'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA', 'AvgCAHH', 'AvgCAHA', 'HGls', 'HAssts', 'HGlsPK', 'HxG', 'HNpxG', 'HxAG',
          'HPoss', 'HPrg', 'HPrgP', 'HGKS%', 'HGKCS%', 'HSoT', 'Hg-xG', 'HCmp', 'HCmp%', 'HPrgDist', 'HXa', 'HKP', 'HPPA', 'HCrsPA',
          'HTB', 'HCrs', 'HCK', 'HSCA', 'HGCA', 'HTkIW', 'HLost', 'HBlocks', 'HPass', 'HInt', 'HClr', 'HAttPen', 'HSucc%', 'HCPA',
          'HDis', 'HPrgR', 'HOnG', 'HOnGA', 'HOnxG', 'HOnxGA', 'AStr', 'ASOH', 'ASOA', 'ASAH', 'ASAA', 'ASDH', 'ASDA', 'AO', 'AA',
          'AM', 'AD', 'ADP', 'AIP', 'ASAAg', 'ATAAg', 'AGls', 'AAssts', 'AGlsPK', 'AxG', 'ANpxG', 'AxAG', 'APoss', 'APrg', 'APrgP',
          'AGKS%', 'AGKCS%', 'ASoT', 'Ag-xG', 'ACmp', 'ACmp%', 'APrgDist', 'AXa', 'AKP', 'APPA', 'ACrsPA', 'ATB', 'ACrs', 'ACK',
          'ASCA', 'AGCA', 'ATkIW', 'ALost', 'ABlocks', 'APass', 'AInt', 'AClr','AAttPen', 'ASucc%', 'ACPA', 'ADis', 'APrgR', 'AOnG',
          'AOnGA', 'AOnxG', 'AOnxGA'], axis=1)
data_basic.head(1)

In [62]:
X = data3[['Date', 'Time', 'TeamH', 'TeamA', 'FTR', 'HTR', 'Referee', 'HStr', 'HSOH', 'HSOA', 'HSAH', 'HSAA', 'HSDH',
          'HSDA', 'HO', 'HA', 'HM', 'HD', 'HSAAg', 'HATAAg']].copy()
y = data3['FTHG']

In [63]:
X = X.drop(columns = ['Date', 'Time', 'FTR', 'HTR'])

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321)

In [65]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse_output = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True))
    ]
)

pipe.fit(X_train, y_train)

Iteration 1, loss = 3.10481765
Iteration 2, loss = 2.82715397
Iteration 3, loss = 2.58475998
Iteration 4, loss = 2.35654138
Iteration 5, loss = 2.14293075
Iteration 6, loss = 1.95222389
Iteration 7, loss = 1.77784655
Iteration 8, loss = 1.61962782
Iteration 9, loss = 1.47625944
Iteration 10, loss = 1.35022535
Iteration 11, loss = 1.24344469
Iteration 12, loss = 1.14939790
Iteration 13, loss = 1.07560238
Iteration 14, loss = 1.01019968
Iteration 15, loss = 0.95998682
Iteration 16, loss = 0.92634313
Iteration 17, loss = 0.90141594
Iteration 18, loss = 0.88627731
Iteration 19, loss = 0.87890934
Iteration 20, loss = 0.87277142
Iteration 21, loss = 0.87060085
Iteration 22, loss = 0.86631461
Iteration 23, loss = 0.85993089
Iteration 24, loss = 0.85279853
Iteration 25, loss = 0.84357835
Iteration 26, loss = 0.83404001
Iteration 27, loss = 0.82390657
Iteration 28, loss = 0.81318760
Iteration 29, loss = 0.80316247
Iteration 30, loss = 0.79384269
Iteration 31, loss = 0.78469839
Iteration 32, los

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr', MLPRegressor(verbose=True))])

In [66]:
print(f'R2 Score: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2 Score: -0.11138053118139601
MAE: 1.1665392817190168


In [67]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (100, 100, 100),
                            max_iter = 1000))
    ]
)

pipe.fit(X_train, y_train)

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 1, loss = 2.17574149
Iteration 2, loss = 1.77089816
Iteration 3, loss = 1.41055715
Iteration 4, loss = 1.11662019
Iteration 5, loss = 0.94429331
Iteration 6, loss = 0.92405857
Iteration 7, loss = 0.97607745
Iteration 8, loss = 0.97323760
Iteration 9, loss = 0.90905867
Iteration 10, loss = 0.84771494
Iteration 11, loss = 0.82264042
Iteration 12, loss = 0.82134117
Iteration 13, loss = 0.81997563
Iteration 14, loss = 0.81061730
Iteration 15, loss = 0.79080623
Iteration 16, loss = 0.76334066
Iteration 17, loss = 0.74127138
Iteration 18, loss = 0.72678734
Iteration 19, loss = 0.72237644
Iteration 20, loss = 0.71332971
Iteration 21, loss = 0.69435823
Iteration 22, loss = 0.67241306
Iteration 23, loss = 0.65494180
Iteration 24, loss = 0.64141341
Iteration 25, loss = 0.63031458
Iteration 26, loss = 0.61319574
Iteration 27, loss = 0.59148235
Iteration 28, loss = 0.57186653
Iteration 29, loss = 0.55606558
Iteration 30, loss = 0.53743623
Iteration 31, loss = 0.52151991
Iteration 32, los

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(hidden_layer_sizes=(100, 100, 100), max_iter=1000,
                              verbose=True))])

In [68]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: -0.4727349819670956
MAE: 1.3395191728460158


In [69]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (60, 60, 60),
                            alpha = 24))
    ]
)

pipe.fit(X_train, y_train)

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 1, loss = 18.13307176
Iteration 2, loss = 17.52603750
Iteration 3, loss = 16.95130594
Iteration 4, loss = 16.43782117
Iteration 5, loss = 15.95748829
Iteration 6, loss = 15.51094567
Iteration 7, loss = 15.10646493
Iteration 8, loss = 14.73615202
Iteration 9, loss = 14.39831015
Iteration 10, loss = 14.06945499
Iteration 11, loss = 13.74786975
Iteration 12, loss = 13.42572374
Iteration 13, loss = 13.10541361
Iteration 14, loss = 12.79006757
Iteration 15, loss = 12.47912843
Iteration 16, loss = 12.18010169
Iteration 17, loss = 11.88815356
Iteration 18, loss = 11.60585390
Iteration 19, loss = 11.32870283
Iteration 20, loss = 11.05976308
Iteration 21, loss = 10.79596285
Iteration 22, loss = 10.53810841
Iteration 23, loss = 10.28561942
Iteration 24, loss = 10.03938826
Iteration 25, loss = 9.79864270
Iteration 26, loss = 9.56399627
Iteration 27, loss = 9.33582004
Iteration 28, loss = 9.11313122
Iteration 29, loss = 8.89542033
Iteration 30, loss = 8.68327095
Iteration 31, loss = 8.47

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(alpha=24, hidden_layer_sizes=(60, 60, 60),
                              verbose=True))])

In [70]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: 0.06984203797915756
MAE: 1.141088275832193


In [71]:
X = data3[['Date', 'Time', 'TeamH', 'TeamA', 'FTR', 'HTR', 'Referee', 'HStr', 'HSOH', 'HSOA', 'HSAH', 'HSAA', 'HSDH',
          'HSDA', 'HO', 'HA', 'HM', 'HD', 'HSAAg', 'HATAAg']].copy()
y = data3['FTAG']

In [72]:
X = X.drop(columns = ['Date', 'Time', 'FTR', 'HTR'])

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321)

In [74]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse_output = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True))
    ]
)

pipe.fit(X_train, y_train)

Iteration 1, loss = 1.65667037
Iteration 2, loss = 1.50007621
Iteration 3, loss = 1.36108155
Iteration 4, loss = 1.24385824
Iteration 5, loss = 1.14706807
Iteration 6, loss = 1.07679488
Iteration 7, loss = 1.01361665
Iteration 8, loss = 0.96652794
Iteration 9, loss = 0.93613401
Iteration 10, loss = 0.91246832
Iteration 11, loss = 0.89380937
Iteration 12, loss = 0.87789876
Iteration 13, loss = 0.86068468
Iteration 14, loss = 0.84417858
Iteration 15, loss = 0.82516700
Iteration 16, loss = 0.80628721
Iteration 17, loss = 0.78569781
Iteration 18, loss = 0.76630454
Iteration 19, loss = 0.74943469
Iteration 20, loss = 0.73085345
Iteration 21, loss = 0.71499490
Iteration 22, loss = 0.69975311
Iteration 23, loss = 0.68507664
Iteration 24, loss = 0.67112588
Iteration 25, loss = 0.65813966
Iteration 26, loss = 0.64512292
Iteration 27, loss = 0.63225539
Iteration 28, loss = 0.61894533
Iteration 29, loss = 0.60821158
Iteration 30, loss = 0.59540061
Iteration 31, loss = 0.58451116
Iteration 32, los

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr', MLPRegressor(verbose=True))])

In [75]:
print(f'R2 Score: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2 Score: -0.3014912539919947
MAE: 1.0251487466885199


In [78]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (100, 100, 100),
                            max_iter = 1000))
    ]
)

pipe.fit(X_train, y_train)

Iteration 1, loss = 1.21957361
Iteration 2, loss = 0.99405852
Iteration 3, loss = 0.83173028
Iteration 4, loss = 0.75705291
Iteration 5, loss = 0.76056307
Iteration 6, loss = 0.76959531
Iteration 7, loss = 0.74013842
Iteration 8, loss = 0.69414045
Iteration 9, loss = 0.66300709
Iteration 10, loss = 0.64132766
Iteration 11, loss = 0.63063753
Iteration 12, loss = 0.61791946
Iteration 13, loss = 0.60087490
Iteration 14, loss = 0.57948374
Iteration 15, loss = 0.55706378
Iteration 16, loss = 0.53666208
Iteration 17, loss = 0.52618222
Iteration 18, loss = 0.50966915
Iteration 19, loss = 0.48970657
Iteration 20, loss = 0.46773923
Iteration 21, loss = 0.45252233
Iteration 22, loss = 0.43810871
Iteration 23, loss = 0.41751410
Iteration 24, loss = 0.39808920
Iteration 25, loss = 0.38234535
Iteration 26, loss = 0.36749915
Iteration 27, loss = 0.34682863
Iteration 28, loss = 0.33069850
Iteration 29, loss = 0.31661955
Iteration 30, loss = 0.29706211
Iteration 31, loss = 0.28367991
Iteration 32, los

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(hidden_layer_sizes=(100, 100, 100), max_iter=1000,
                              verbose=True))])

In [79]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: -0.1427948015473175
MAE: 0.9630138297809253


In [80]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (60, 60, 60),
                            alpha = 24))
    ]
)

pipe.fit(X_train, y_train)

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 1, loss = 16.76213140
Iteration 2, loss = 16.31397586
Iteration 3, loss = 15.88825966
Iteration 4, loss = 15.47444118
Iteration 5, loss = 15.07233718
Iteration 6, loss = 14.68223908
Iteration 7, loss = 14.30235051
Iteration 8, loss = 13.93335986
Iteration 9, loss = 13.57519325
Iteration 10, loss = 13.22487367
Iteration 11, loss = 12.88461693
Iteration 12, loss = 12.55215298
Iteration 13, loss = 12.23048407
Iteration 14, loss = 11.91529032
Iteration 15, loss = 11.60856727
Iteration 16, loss = 11.30934305
Iteration 17, loss = 11.01799057
Iteration 18, loss = 10.73377899
Iteration 19, loss = 10.45655502
Iteration 20, loss = 10.18708940
Iteration 21, loss = 9.92372486
Iteration 22, loss = 9.66738648
Iteration 23, loss = 9.41746338
Iteration 24, loss = 9.17412390
Iteration 25, loss = 8.93730771
Iteration 26, loss = 8.70615791
Iteration 27, loss = 8.48119433
Iteration 28, loss = 8.26194103
Iteration 29, loss = 8.04867728
Iteration 30, loss = 7.84092701
Iteration 31, loss = 7.638329

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(alpha=24, hidden_layer_sizes=(60, 60, 60),
                              verbose=True))])

In [81]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: -0.01608328430147421
MAE: 0.9497084008139123
